<a href="https://colab.research.google.com/github/politeles/audio_classification/blob/main/VADSAudioClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing an audio classifier for the VADS dataset
the VADS dataset contains multiple sample of audio sounds of the violin played in different techniques.
We will train a classifier using hugging face transformers library.
But first we have to download and preprocess the data, and perform operations like normalization

In [ ]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9war2eq9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-9war2eq9
  Resolved https://github.com/huggingface/transformers to commit 7eecdf2a8650306ed5fbb6150c64f99f587e004d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10627281 sha256=04d7937abb8ecd9293804075b3ea16f43c6924ae6970a7289e4fba3b66453d2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-oy9lygea/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [ ]:
pip install datasets[audio]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whic

In [ ]:
from datasets import load_dataset

In [ ]:
vads = load_dataset("politeles/vads")

NameError: name 'load_dataset' is not defined

In [ ]:
vads

DatasetDict({
    train: Dataset({
        features: ['audio', 'technique', 'microphone', 'string', 'note'],
        num_rows: 2515
    })
})

# Undersampling the dataset

In [ ]:
from collections import Counter

technique_counts = Counter(vads["train"]["technique"])
underrepresented_class = min(technique_counts, key=technique_counts.get)
undersample_count = technique_counts[underrepresented_class]

In [ ]:
import pandas as pd
from datasets import Dataset

# Create a Pandas DataFrame from the dataset
df = pd.DataFrame(vads["train"])

# Undersample the DataFrame
undersampled_df =  df.groupby("technique", group_keys=False).apply(lambda x: x.sample(n=undersample_count) if len(x) > undersample_count else x).reset_index(drop=True)

# Convert the undersampled DataFrame back to a Hugging Face dataset
undersampled_dataset = Dataset.from_pandas(undersampled_df)

<ipython-input-26-e89944c44b43>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  undersampled_df =  df.groupby("technique", group_keys=False).apply(lambda x: x.sample(n=undersample_count) if len(x) > undersample_count else x).reset_index(drop=True)


In [ ]:
undersampled_dataset

Dataset({
    features: ['audio', 'technique', 'microphone', 'string', 'note'],
    num_rows: 753
})

# Generate train test split

In [ ]:
undersampled_dataset = undersampled_dataset.train_test_split(test_size=0.2)

NameError: name 'undersampled_dataset' is not defined

In [ ]:
undersampled_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'technique', 'microphone', 'string', 'note'],
        num_rows: 602
    })
    test: Dataset({
        features: ['audio', 'technique', 'microphone', 'string', 'note'],
        num_rows: 151
    })
})

## Translate the string characteristics: technique and microphone

# Preprocessing data

In [ ]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [ ]:
sampling_rate = feature_extractor.sampling_rate

In [ ]:
sampling_rate

16000

## Resample data

In [ ]:
from datasets import Audio

undersampled_dataset = undersampled_dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

## Normalize audio

In [ ]:
import numpy as np

sample = undersampled_dataset["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: -1.65e-05, Variance: 0.00275


In [ ]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -3.58e-09, Variance: 1.0


## Preprocess function
Truncates the audio to 1 second in the sampled audio rate

In [ ]:
max_duration = 1.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [ ]:
vads_encoded = undersampled_dataset.map(
    preprocess_function,
    remove_columns=["audio", "microphone","string","note"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
vads_encoded

Map:   0%|          | 0/602 [00:00<?, ? examples/s]

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['technique', 'input_values', 'attention_mask'],
        num_rows: 602
    })
    test: Dataset({
        features: ['technique', 'input_values', 'attention_mask'],
        num_rows: 151
    })
})

To allow the model to process we need to rename the column "technique" to "label"

In [ ]:
vads_encoded = vads_encoded.rename_column("technique","label")

NameError: name 'vads_encoded' is not defined

In [ ]:
vads_encoded['train']['label'][10]

2

# Fine tunning a model


In [ ]:
id2label_fn = vads["train"].features["technique"].int2str

NameError: name 'vads' is not defined

In [ ]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(vads["train"].features["technique"].names))
}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1576: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=vads_encoded["train"],
    eval_dataset=vads_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-54-56fee3745cf2>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.700800,1.600971,0.549669
2,0.891800,0.934632,0.695364
3,0.680200,0.673445,0.781457
4,0.329100,0.480321,0.854305
5,0.260900,0.347315,0.880795
6,0.106100,0.243921,0.927152
7,0.125200,0.212661,0.953642
8,0.084000,0.197994,0.940397
9,0.037400,0.200539,0.940397
10,0.043100,0.198766,0.940397


TrainOutput(global_step=760, training_loss=0.5182216224701781, metrics={'train_runtime': 4134.2548, 'train_samples_per_second': 1.456, 'train_steps_per_second': 0.184, 'total_flos': 1.369036917260532e+16, 'train_loss': 0.5182216224701781, 'epoch': 10.0})

In [ ]:
trainer.save_model(f"{model_name}-finetuned-vads")

No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
# Update model card
trainer.create_model_card(
    tags=["audio-classification", "vads", "violin"],
    finetuned_from=model_id,
    dataset_tags="politeles/vads",
    dataset="vads",  # Or your specific dataset name
    # You can add more metadata here as needed, like license, model_name, tasks, etc.
)

In [ ]:
kwargs = {
    "dataset_tags": "politeles/vads",
    "dataset": "VADS",
    "model_name": f"{model_name}-finetuned-vads",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [ ]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/politeles/distilhubert-finetuned-gtzan/commit/bd97b293eb57920ea047b93163502d0b6643a231', commit_message='End of training', commit_description='', oid='bd97b293eb57920ea047b93163502d0b6643a231', pr_url=None, repo_url=RepoUrl('https://huggingface.co/politeles/distilhubert-finetuned-gtzan', endpoint='https://huggingface.co', repo_type='model', repo_id='politeles/distilhubert-finetuned-gtzan'), pr_revision=None, pr_num=None)